<a href="https://colab.research.google.com/github/saeidsaadatigero/AI_ML_Computer-Vision/blob/main/23_Logistic_RegressionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Logistic Regression روی دیتاست MNIST
# =====================================

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 1️⃣ آماده‌سازی دیتاست
transform = transforms.Compose([
    transforms.ToTensor(),                  # تبدیل عکس به Tensor
    transforms.Normalize((0.5,), (0.5,))    # نرمال‌سازی بین -1 و 1
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=1000, shuffle=False)

# 2️⃣ تعریف مدل
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim=28*28, num_classes=10):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, num_classes)  # فقط یک لایه خطی

    def forward(self, x):
        x = x.view(-1, 28*28)  # باز کردن تصویر از 2D به 1D
        outputs = self.linear(x)  # خروجی خام (logits)
        return outputs             # بدون softmax (CrossEntropy خودش داره)

model = LogisticRegressionModel()
print(model)

# 3️⃣ تنظیم Loss و Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 4️⃣ حلقه آموزش (Training Loop)
num_epochs = 5
for epoch in range(num_epochs):
    total_loss = 0
    for images, labels in train_loader:
        optimizer.zero_grad()       # صفر کردن گرادیان‌ها
        outputs = model(images)     # پیش‌بینی مدل
        loss = criterion(outputs, labels)  # محاسبه loss
        loss.backward()             # محاسبه گرادیان‌ها
        optimizer.step()            # به‌روزرسانی وزن‌ها
        total_loss += loss.item()

    print(f"دوره {epoch+1}/{num_epochs}, میانگین خطا: {total_loss/len(train_loader):.4f}")

# 5️⃣ ارزیابی مدل روی داده تست
correct = 0
total = 0
with torch.no_grad():  # غیرفعال کردن محاسبات گرادیان
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)  # انتخاب بیشترین احتمال
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"\n✅ دقت مدل روی داده تست: {100 * correct / total:.2f}%")


100%|██████████| 9.91M/9.91M [00:00<00:00, 58.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.76MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 15.0MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.69MB/s]


LogisticRegressionModel(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)
دوره 1/5, میانگین خطا: 0.6087
دوره 2/5, میانگین خطا: 0.3872
دوره 3/5, میانگین خطا: 0.3530
دوره 4/5, میانگین خطا: 0.3354
دوره 5/5, میانگین خطا: 0.3242

✅ دقت مدل روی داده تست: 91.38%
